In [1]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
import matplotlib.pyplot as plt
import cv2

##### New Function Created for HSV Color Space

In [2]:
def Detection_Using_HSVColorspace(image, x_min, y_min, width, height):
    
    ##getting region of interest
    region_of_interest = image[y_min: y_min+height, x_min:x_min+width]
    
    ##converting region of interest from BGR to HSV Format
    region_of_int_hsv = cv2.cvtColor(region_of_interest, cv2.COLOR_BGR2HSV)
    
    
    ##getting the histogram for hue channel of region of interest in hsv
    histogram_regio_of_int = cv2.calcHist([region_of_int_hsv], [0], None, [180], [0, 180])
    
    
    ##converting the image to hsv format
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    ##creating the mask from the hsv image and region of interest histogram
    hsv_mask = cv2.calcBackProject([image_hsv], [0], histogram_regio_of_int, [0, 180], 1)
    
    ##termination criteria has been set to 10 iterations for mean shift algorithm, 
    ## moving by 1point on the image and mean shift operation is stopped with error 
    ## termed by 'TERM_CRITERIA_EPS'
    metric_used = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT, 10, 1) 
    
    
    ##applying the meanshift with the mask
    ret, detector_win = cv2.meanShift(hsv_mask, (x_min, y_min, width, height),  metric_used)
    
    ##getting the coordinates of the updated bounding box/window
    x, y, w, h = detector_win
    
    return x, y, x+w, y+h

###### Loading the Training Dataset Images

In [10]:
import os
import json
def loading_of_dataset(directory):
    imgs_dir = os.listdir(directory)
    
    
    count = 0
    
    ##going through annotations folder in valid directory
    train_annotations_dir = 'train_annotations'
    
    ##going through the file and reading loading the contents
    with open(train_annotations_dir , 'r') as f:
         train_annotations_given = json.load(f)

    
    
    
    
    for label in imgs_dir:
        img_dir = os.path.join(directory, label)##taking the full image directory
        image = cv2.imread(img_dir)
        img_rgb = image[:,:,::-1]
        
        ###getting the coordinates of the bounding box of the respective images in annotation file
        x_min, y_min, width, height = train_annotations_given[count]['bbox']
        
        ###getting the coordinates detected from the function
        min_x, min_y, max_x, max_y =  Detection_Using_HSVColorspace(image, x_min, y_min, width, height)
        
        cropped_image = image[min_y:max_y, min_x:max_x]
        
        cv2.imwrite('MeanShift_Train_Images/{}.jpg'.format(count), cropped_image)
        count = count + 1
        

In [11]:
train_directory = 'Penguins vs Turtles/archive/train/train'
loading_of_dataset(train_directory)

###### Loading the Validation Dataset Images

In [3]:
import os
import json
def loading_of_dataset(directory):
    imgs_dir = os.listdir(directory)
    
    
    count = 0
    imgs_dir = imgs_dir[0:]
    predicted_coordinates = []
    
    ##going through annotations folder in valid directory
    valid_annotations_dir = 'valid_annotations'
    
    ##going through the file and reading loading the contents
    with open(valid_annotations_dir , 'r') as f:
         valid_annotations_given = json.load(f)

    
    
    
    
    for label in imgs_dir:
        img_dir = os.path.join(directory, label)##taking the full image directory
        image = cv2.imread(img_dir)
        img_rgb = image[:,:,::-1]
        
        
        x_min, y_min, width, height = valid_annotations_given[count]['bbox']
        
        min_x, min_y, max_x, max_y =  Detection_Using_HSVColorspace(image, x_min, y_min, width, height)
        
        predicted_coordinates.append([ min_x, min_y, max_x, max_y])
        
        cropped_image = image[min_y:max_y, min_x:max_x]
        
        cv2.imwrite('MeanShift_Valid_Images/{}.jpg'.format(count), cropped_image)
        count = count + 1
        
    return predicted_coordinates        

In [4]:
valid_directory = 'Penguins vs Turtles/archive/valid/valid'
list_of_coords = loading_of_dataset(valid_directory)

In [5]:
list_of_coords

[[227, 83, 525, 608],
 [211, 198, 639, 457],
 [0, 233, 512, 500],
 [128, 178, 585, 532],
 [168, 135, 464, 604],
 [0, 121, 445, 592],
 [53, 161, 491, 483],
 [178, 242, 432, 518],
 [375, 228, 564, 543],
 [364, 200, 505, 446],
 [145, 173, 630, 533],
 [123, 271, 474, 387],
 [131, 39, 630, 421],
 [126, 414, 519, 565],
 [178, 109, 464, 592],
 [363, 187, 618, 406],
 [103, 167, 599, 433],
 [50, 167, 536, 640],
 [107, 293, 263, 366],
 [114, 205, 625, 428],
 [127, 101, 523, 506],
 [169, 311, 452, 538],
 [0, 228, 414, 436],
 [136, 238, 494, 523],
 [33, 138, 291, 526],
 [17, 202, 621, 405],
 [4, 179, 490, 529],
 [252, 158, 640, 376],
 [206, 256, 439, 372],
 [183, 231, 542, 454],
 [61, 127, 514, 580],
 [171, 287, 464, 430],
 [64, 189, 470, 544],
 [75, 274, 423, 418],
 [241, 321, 356, 384],
 [155, 161, 489, 486],
 [66, 186, 609, 460],
 [249, 281, 343, 455],
 [183, 142, 325, 505],
 [168, 170, 640, 404],
 [84, 251, 490, 573],
 [232, 230, 585, 405],
 [306, 252, 510, 509],
 [99, 61, 582, 541],
 [252, 20

In [6]:
valid_annotations_dir = 'valid_annotations'

with open(valid_annotations_dir, 'r') as f:
    valid_annotations_given = json.load(f)

In [7]:
actual_coordinates = []

for i in range(len(list_of_coords)):
    ##getting x_max = width + x_min
    x_min, y_min, width, height = valid_annotations_given[i]['bbox']
    actual_coordinates.append([x_min, y_min, x_min + width, y_min + height])

In [8]:
actual_coordinates

[[227, 93, 525, 618],
 [211, 198, 639, 457],
 [0, 243, 512, 510],
 [128, 178, 585, 532],
 [168, 131, 464, 600],
 [6, 121, 451, 592],
 [63, 161, 501, 483],
 [188, 252, 442, 528],
 [375, 237, 564, 552],
 [364, 223, 505, 469],
 [155, 173, 640, 533],
 [133, 271, 484, 387],
 [141, 39, 640, 421],
 [126, 414, 519, 565],
 [178, 119, 464, 602],
 [363, 187, 618, 406],
 [103, 167, 599, 433],
 [60, 167, 546, 640],
 [164, 284, 320, 357],
 [114, 205, 625, 428],
 [137, 111, 533, 516],
 [159, 321, 442, 548],
 [0, 228, 414, 436],
 [156, 248, 514, 533],
 [43, 138, 301, 526],
 [35, 209, 639, 412],
 [4, 169, 490, 519],
 [252, 168, 640, 386],
 [223, 256, 456, 372],
 [193, 231, 552, 454],
 [61, 127, 514, 580],
 [171, 297, 464, 440],
 [76, 179, 482, 534],
 [75, 306, 423, 450],
 [231, 333, 346, 396],
 [155, 161, 489, 486],
 [56, 186, 599, 460],
 [258, 294, 352, 468],
 [193, 152, 335, 515],
 [168, 180, 640, 414],
 [94, 251, 500, 573],
 [232, 230, 585, 405],
 [316, 252, 520, 509],
 [99, 81, 582, 561],
 [252, 21

In [9]:
import math

def estimate_centre_pos(coords_list):
    
    ##getting the following coordinates
    x_min, y_min, x_max, y_max = coords_list
    
    ##estimating center pos of x-coordinate
    x_center = (x_min + x_max)/2 
    
    ##estimating center pos of y-coordinate
    y_center = (y_min + y_max)/2 
    
    
    return x_center, y_center   


def Euclid_Dist_coords(pred_list, actual_list):
    
    ##getting centre positions of predicted bounding box
    center_x_pred, center_y_pred = estimate_centre_pos(pred_list)
    
    ##getting center positions of actual bounding box
    center_x_actual, center_y_actual = estimate_centre_pos(actual_list)
    
    ##calculating the euclidean distance
    centre_distance = math.sqrt((center_x_pred - center_x_actual)**2 + (center_y_pred - center_y_actual)**2)
    
    return centre_distance  

In [10]:
measures_dist = []
for i, j in zip(list_of_coords, actual_coordinates):
    dist = Euclid_Dist_coords(i, j)
    measures_dist.append(dist)

In [11]:
measures_dist

[10.0,
 0.0,
 10.0,
 0.0,
 4.0,
 6.0,
 10.0,
 14.142135623730951,
 9.0,
 23.0,
 10.0,
 10.0,
 10.0,
 0.0,
 10.0,
 0.0,
 0.0,
 10.0,
 57.706152185014034,
 0.0,
 14.142135623730951,
 14.142135623730951,
 0.0,
 22.360679774997898,
 10.0,
 19.313207915827967,
 10.0,
 10.0,
 17.0,
 10.0,
 0.0,
 10.0,
 15.620499351813308,
 32.0,
 15.620499351813308,
 0.0,
 10.0,
 15.811388300841896,
 14.142135623730951,
 10.0,
 10.0,
 0.0,
 10.0,
 20.0,
 10.0,
 22.360679774997898,
 14.142135623730951,
 17.0,
 14.142135623730951,
 14.142135623730951,
 1.0,
 18.0,
 0.0,
 10.0,
 21.470910553583888,
 10.0,
 68.0294054067798,
 0.0,
 0.0,
 0.0,
 10.0,
 0.0,
 10.0,
 20.0,
 0.0,
 0.0,
 3.605551275463989,
 17.72004514666935,
 10.44030650891055,
 14.142135623730951,
 14.142135623730951,
 6.0]

In [12]:
mean_dist_btwn_centres = sum(measures_dist)/len(measures_dist)
print("Mean Distance between Center of Prdicted and Actual is: ", mean_dist_btwn_centres)

Mean Distance between Center of Prdicted and Actual is:  11.254702030004061


In [13]:
def calculate_Standard_dev(data, mean):
    
    #mean = calculate_mean(data)
    
    sum_squared_diff = 0
    for x in data:
        sq_diff = (x - mean) ** 2
        sum_squared_diff = sum_squared_diff + sq_diff
        
        
    variance = sum_squared_diff / len(data)
    
    standard_dev = math.sqrt(variance)

    return  standard_dev

In [14]:
standard_dev_btwn_centers = calculate_Standard_dev(measures_dist, mean_dist_btwn_centres)
print("Standard Deviation between Center of Prdicted and Actual is: ", standard_dev_btwn_centers)

Standard Deviation between Center of Prdicted and Actual is:  11.347344965734406


In [15]:
def estimate_intersect_area(pred_cords, actual_cords):
    
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_cords
    
    x_min_actual, y_min_actual, x_max_actual, y_max_actual = actual_cords
    
    ##common area in x-axis
    commomn_x_area = max(0, (min(x_max_pred, x_max_actual) - max(x_min_pred,  x_min_actual)))
    
    ##common-area in y-axis
    commomn_y_area = max(0, (min(y_max_pred, y_max_actual) - max(y_min_pred,  y_min_actual)))
    
    ##getting the common area between two images
    intersection_area = commomn_x_area * commomn_y_area

    
    return intersection_area

In [16]:
def estimate_union_area(pred_cords, actual_cords):
    
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_cords
    
    x_min_actual, y_min_actual, x_max_actual, y_max_actual = actual_cords
    
    
    area_pred =  (x_max_pred -  x_min_pred) * (y_max_pred - y_min_pred)
    
    area_actual = (x_max_actual -  x_min_actual) * (y_max_actual - y_min_actual)
    
    area_estimated_union = area_pred + area_actual - estimate_intersect_area(pred_cords, actual_cords)
    
    return area_estimated_union

In [17]:
def estimate_iou_calculate(pred_bbox, actual_bbox):
    
    area_intersected = estimate_intersect_area(pred_bbox, actual_bbox)
    area_union = estimate_union_area(pred_bbox, actual_bbox)
    estimate_iou = area_intersected / area_union
    return estimate_iou

In [18]:
measures_iou = []
for m, n in zip(list_of_coords, actual_coordinates):
    iou_img = estimate_iou_calculate(m, n)
    measures_iou .append(iou_img)

In [19]:
measures_iou

[0.9626168224299065,
 1.0,
 0.927797833935018,
 1.0,
 0.9830866807610994,
 0.9733924611973392,
 0.9553571428571429,
 0.8618931265271433,
 0.9444444444444444,
 0.828996282527881,
 0.9595959595959596,
 0.9445983379501385,
 0.9607072691552063,
 1.0,
 0.9594320486815415,
 1.0,
 1.0,
 0.9596774193548387,
 0.3854014598540146,
 1.0,
 0.9059956028284509,
 0.855576898080617,
 1.0,
 0.8365583655836558,
 0.9253731343283582,
 0.8810137457044673,
 0.9444444444444444,
 0.9122807017543859,
 0.864,
 0.94579945799458,
 1.0,
 0.869281045751634,
 0.8923390008534103,
 0.6363636363636364,
 0.5862068965517241,
 1.0,
 0.9638336347197106,
 0.719241078467441,
 0.8247663551401869,
 0.9180327868852459,
 0.9519230769230769,
 1.0,
 0.9065420560747663,
 0.92,
 0.888268156424581,
 0.7914064451661255,
 0.8725900714301709,
 0.8645418326693227,
 0.8976888128934706,
 0.8049866912923027,
 0.9965753424657534,
 0.9076923076923077,
 1.0,
 0.9173553719008265,
 0.7758421473646787,
 0.9456521739130435,
 0.16880126636926177,
 1

In [20]:
mean_iou = sum(measures_iou) / len(measures_iou)
mean_iou

print("Mean IOU between Center of Prdicted and Actual is: ", mean_iou)

Mean IOU between Center of Prdicted and Actual is:  0.9014799184790563


In [21]:
standard_dev_btwn_centers_iou = calculate_Standard_dev(measures_iou, mean_iou)
print("Standard Deviation between Center of Prdicted and Actual is: ", standard_dev_btwn_centers_iou)

Standard Deviation between Center of Prdicted and Actual is:  0.13499728466406788
